""" IGNORE THIS FOR NOW !!!!! """
class SensorEventClass:
    """WIP"""

    #ZIEL: DEN GANZEN SPAß FÜR EINEN SENSOR HINBEKOMMEN.
    # --> Funktioniert es für einen, funktioniert es für alle
    
    # definitions
    __movement_west = -34
    movement_WE = 34
    __movement_north = -1
    movement_NS = 1
    date_format = "%Y-%m-%d %H:%M:%S"
    radar_filepath = r"Data\\radar_data"
    
    def __init__(self, sensor_EID: str = "Test", 
                 #grid_width: int = 1, 
                 #t_width: int = 1, 
                 test = False,
                 **kwargs):
        
        # Grundelegende Variablen 
        print("initializing", end="...")
        sensor_id_df = pd.read_csv(r"Data\\sensors_nr_deveui.csv", sep=";", usecols=["devEui", "Stationsnr"])
        self.devEUI_to_Stationsnr = sensor_id_df.set_index("devEui").to_dict()["Stationsnr"]
        self.Stationsnr_to_devEUI = sensor_id_df.set_index("Stationsnr").to_dict()["devEui"]
        self.__sensor_EID = sensor_EID
        #self.__grid_width = grid_width
        #self.__t_width = t_width
        
        print("done")
        
        
        # test conditions:
        if test:
            self.set_event_timeframe("2024-05-26 12:00:00", "2024-05-26 14:20:00")
            self.__sensor_EID = "E1E54"
            import_radfiles = "n"
        
        # set timeframe according to class call
        elif "start" in kwargs.keys() and "end" in kwargs.keys():
            try:
                self.set_timeframe(kwargs["start"], kwargs["end"])
            except Exception as e:
                print("\n" + e)
                self.input_timeframe()
        
        # get timeframe input
        else:
            self.input_timeframe()
        
        # set pixel_id and EID
        self.pixel_ID = SENSORS_PIXELID[self.__sensor_EID]
        self.__stationsnr = self.devEUI_to_Stationsnr[self.__sensor_EID]
        self.pixel_addition_grid = self.initialize_grid_array(grid_width)
        
        # import sensor data via sationsnr
        print("importing Sensor-Data", end="...")
        self.__station_df = create_single_sensor_df(self.__stationsnr, SENSOR_DATA_PATH,
                                                    self.__start_datetime, self.__end_datetimel)
        print("done")
        
        
        
        #self.__radar_data = import_radar_data(self.__sensor_EID)
                
            
    def get_station_df(self):
        return self.__station_df

    def print_event_timeframe(self):
        print(f"gegebener Timeframe: {self.__start_timestring} - {self.__end_timestring}")
        print(f"Timeframe Rechnung: {self.__start_datetime} - {self.__end_datetime}")
        
    def get_start_datetime(self): return self.__start_datetime
    
    def get_end_datetime(self): return self.__end_datetime
            
    def set_event_timeframe(self, start, end):
        self.__start_datetime = dt.strptime(start, self.date_format) - td(minutes=5*self.__t_width)
        self.__end_datetime = dt.strptime(end, self.date_format) + td(minutes=5*self.__t_width)
        self.__timerange = pd.date_range(self.__start_datetime, self.__end_datetime, freq="5min")
        self.__start_timestring = start
        self.__end_timestring = end
    
    
    def input_timeframe(self):
        while True:
            print("Start- und Endzeit des Events wurden entweder nicht oder nicht korrekt angegeben.")
            print("Das Korrekte Format ist <YYYY-mm-DD HH:MM:SS>. Bspw (2024-05-10 10:00:00)")
            
            start = input("Bitte geben Sie einen Startzeitpunkt an für das Regenevent an: ")
            end = input("Bitte geben Sie einen Endzeitpunkt an für das Regenevent an: ")
            
            try:
                self.set_event_timeframe(start, end)
                break
            except Exception as e:
                print(e)
                
    def get_sensor_data(self):
        sensor_rawdata = os.path.join(SENSOR_DATA_PATH)

    def initialize_grid_array(self, grid_width):
        self.pixel_addition_grid = []
        self.pixel_grid = []
        
        for i in range(-grid_width, grid_width + 1):
            line = []
            pixelline = []
            for j in range(-grid_width, grid_width + 1):
                line.append(i * self.movement_NS + j * self.movement_WE)
                pixelline.append(self.pixel_ID + i * self.movement_NS + j * self.movement_WE)
                
            self.pixel_grid.append(pixelline)
            self.pixel_addition_grid.append(line)
        
        return self.pixel_addition_grid
    
    # DEPRECATED
    def import_radar_data(self):
        #TODO
        # make list of files
        files_list = [f"hd{timeframe.strftime('%y%m%d%H%M')}.scu" for timeframe in self.__timerange]
        
        # for every file in file List
        for file in files_list:
            filestring = os.path.join(self.radar_filepath, file)
            
            try: 
                ncf = netCDF4.Dataset(filestring, mode="r")
                image_data = ncf.groups["dataset_DXk"].variables["image"][:]
                ncf.close()
            except Exception as e:
                print(e)
                raise e
        
            # Extract and process the specific array sections
            array_result = [image_data[i, 147:201] for i in range(119, 153)]
            transposed_arrays = list(map(list, zip(*array_result)))
            array_result = np.array(transposed_arrays)/12  ##Divide per 12 to give the result in mm
            
            # Flatten the nested list
            list_array = [item for sublist in array_result for item in sublist]
            # get pixeldata and append to corresponding gridposition
            
            self.radar_data_grid = []       # create empty array where radar data dfs get input into
            for i, row in enumerate(self.pixel_grid):
                print(i, row)
                for j, val in enumerate(row):
                    print(f"[{i}:{j}]: {val}")
                    
            
    
                
                

    
            
        
    
        

In [ ]:
""" IGNORE THIS FOR NOW !!!!! """
class SensorEventClass:
    """WIP"""

    #ZIEL: DEN GANZEN SPAß FÜR EINEN SENSOR HINBEKOMMEN.
    # --> Funktioniert es für einen, funktioniert es für alle
    
    # definitions
    __movement_west = -34
    movement_WE = 34
    __movement_north = -1
    movement_NS = 1
    date_format = "%Y-%m-%d %H:%M:%S"
    radar_filepath = r"Data\\radar_data"
    
    def __init__(self, sensor_EID: str = "Test", 
                 #grid_width: int = 1, 
                 #t_width: int = 1, 
                 test = False,
                 **kwargs):
        
        # Grundelegende Variablen 
        print("initializing", end="...")
        sensor_id_df = pd.read_csv(r"Data\\sensors_nr_deveui.csv", sep=";", usecols=["devEui", "Stationsnr"])
        self.devEUI_to_Stationsnr = sensor_id_df.set_index("devEui").to_dict()["Stationsnr"]
        self.Stationsnr_to_devEUI = sensor_id_df.set_index("Stationsnr").to_dict()["devEui"]
        self.__sensor_EID = sensor_EID
        #self.__grid_width = grid_width
        #self.__t_width = t_width
        
        print("done")
        
        
        # test conditions:
        if test:
            self.set_event_timeframe("2024-05-26 12:00:00", "2024-05-26 14:20:00")
            self.__sensor_EID = "E1E54"
            import_radfiles = "n"
        
        # set timeframe according to class call
        elif "start" in kwargs.keys() and "end" in kwargs.keys():
            try:
                self.set_timeframe(kwargs["start"], kwargs["end"])
            except Exception as e:
                print("\n" + e)
                self.input_timeframe()
        
        # get timeframe input
        else:
            self.input_timeframe()
        
        # set pixel_id and EID
        self.pixel_ID = SENSORS_PIXELID[self.__sensor_EID]
        self.__stationsnr = self.devEUI_to_Stationsnr[self.__sensor_EID]
        self.pixel_addition_grid = self.initialize_grid_array(grid_width)
        
        # import sensor data via sationsnr
        print("importing Sensor-Data", end="...")
        self.__station_df = create_single_sensor_df(self.__stationsnr, SENSOR_DATA_PATH,
                                                    self.__start_datetime, self.__end_datetimel)
        print("done")
        
        
        
        #self.__radar_data = import_radar_data(self.__sensor_EID)
                
            
    def get_station_df(self):
        return self.__station_df

    def print_event_timeframe(self):
        print(f"gegebener Timeframe: {self.__start_timestring} - {self.__end_timestring}")
        print(f"Timeframe Rechnung: {self.__start_datetime} - {self.__end_datetime}")
        
    def get_start_datetime(self): return self.__start_datetime
    
    def get_end_datetime(self): return self.__end_datetime
            
    def set_event_timeframe(self, start, end):
        self.__start_datetime = dt.strptime(start, self.date_format) - td(minutes=5*self.__t_width)
        self.__end_datetime = dt.strptime(end, self.date_format) + td(minutes=5*self.__t_width)
        self.__timerange = pd.date_range(self.__start_datetime, self.__end_datetime, freq="5min")
        self.__start_timestring = start
        self.__end_timestring = end
    
    
    def input_timeframe(self):
        while True:
            print("Start- und Endzeit des Events wurden entweder nicht oder nicht korrekt angegeben.")
            print("Das Korrekte Format ist <YYYY-mm-DD HH:MM:SS>. Bspw (2024-05-10 10:00:00)")
            
            start = input("Bitte geben Sie einen Startzeitpunkt an für das Regenevent an: ")
            end = input("Bitte geben Sie einen Endzeitpunkt an für das Regenevent an: ")
            
            try:
                self.set_event_timeframe(start, end)
                break
            except Exception as e:
                print(e)
                
    def get_sensor_data(self):
        sensor_rawdata = os.path.join(SENSOR_DATA_PATH)

    def initialize_grid_array(self, grid_width):
        self.pixel_addition_grid = []
        self.pixel_grid = []
        
        for i in range(-grid_width, grid_width + 1):
            line = []
            pixelline = []
            for j in range(-grid_width, grid_width + 1):
                line.append(i * self.movement_NS + j * self.movement_WE)
                pixelline.append(self.pixel_ID + i * self.movement_NS + j * self.movement_WE)
                
            self.pixel_grid.append(pixelline)
            self.pixel_addition_grid.append(line)
        
        return self.pixel_addition_grid
    
    # DEPRECATED
    def import_radar_data(self):
        #TODO
        # make list of files
        files_list = [f"hd{timeframe.strftime('%y%m%d%H%M')}.scu" for timeframe in self.__timerange]
        
        # for every file in file List
        for file in files_list:
            filestring = os.path.join(self.radar_filepath, file)
            
            try: 
                ncf = netCDF4.Dataset(filestring, mode="r")
                image_data = ncf.groups["dataset_DXk"].variables["image"][:]
                ncf.close()
            except Exception as e:
                print(e)
                raise e
        
            # Extract and process the specific array sections
            array_result = [image_data[i, 147:201] for i in range(119, 153)]
            transposed_arrays = list(map(list, zip(*array_result)))
            array_result = np.array(transposed_arrays)/12  ##Divide per 12 to give the result in mm
            
            # Flatten the nested list
            list_array = [item for sublist in array_result for item in sublist]
            # get pixeldata and append to corresponding gridposition
            
            self.radar_data_grid = []       # create empty array where radar data dfs get input into
            for i, row in enumerate(self.pixel_grid):
                print(i, row)
                for j, val in enumerate(row):
                    print(f"[{i}:{j}]: {val}")
                    
            
    
                
                

    
            
        
    
        